In [ ]:
import h5py, sys, os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm

In [ ]:
sys.path.append('/mnt/cbis/home/benedikt/.local/dragonfly/utils/py_src')

In [ ]:
import writeemc
import detector
import reademc

In [ ]:
path_to_data = "../../data/sparse/" # scratch/sparse on Maxwell
path_to_aux = "../../data/aux/" # scratch/benedikt/aux on Maxwell
path_to_geometry = "../../geometry/"
path_to_recons = "../../data/recons/"

## Reading metadata

In [ ]:
with h5py.File(path_to_aux + "sparse_hits_metadata.h5", "r") as f:
    cells = f['cells'][:]
    score = f['scores'][:]
    train = f['train'][:]

## Reading from 2D EMC classes

In [ ]:
emc_folder = path_to_recons + "melv_0001/"
emc_output = emc_folder + "data/r0/output_040.h5"
emc_photons = emc_folder + 'photons.txt'
emc_detector = path_to_recons + "det/det_lowq7.h5"

In [ ]:
photons_list = list(np.loadtxt(emc_photons, dtype=str))
photons_list = [emc_folder + p for p in photons_list]

In [ ]:
det = detector.Detector(emc_detector, mask_flag=True)
emc = reademc.EMCReader(photons_list, det)

In [ ]:
with h5py.File(emc_output,'r') as f:
    occupancies = f['occupancies'][:]
    likelihood = f['likelihood'][:]
    orientations = f['orientations'][:]
    images = f['intens'][:]

In [ ]:
modes = orientations % occupancies.shape[1]
ndata = len(modes)

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=5, figsize=(20,12), dpi=200)
for i in range(15):
    c,r = i//5, i%5
    axes[c,r].axis('off')
    nclass = (modes==i).sum()
    axes[c,r].set_title("%d/%d [%.1f%%]" %(nclass,ndata, nclass/ndata*100.))
    axes[c,r].imshow(images[i], norm=colors.LogNorm(vmin=0.001, vmax=3), cmap='cividis')
plt.show()

In [ ]:
rings = (modes == 13)
cellid_rings  = cells[rings]
scores_rings  = score[rings]
trainid_rings = train[rings]
index_rings   = np.where(rings)[0]

In [ ]:
images_rings  = np.array([emc.get_frame(i) for i in index_rings])
modules_rings = np.array([emc.get_frame(i, raw=True) for i in index_rings])

In [ ]:
nrphot_rings = np.array([images_rings[i][np.bool8(det.mask)].sum() for i in range(images_rings.shape[0])])

In [ ]:
plt.figure(figsize=(10,5), dpi=100)
plt.title("All hits (blue) vs. %d ring-like hits (red)" %rings.sum())
plt.plot(train-train[0], cells, marker='.', color='lightblue', lw=0, ms=1, alpha=0.1)
plt.plot(trainid_rings-train[0], cellid_rings, marker='.', color='r', lw=0, ms=1, alpha=1)
plt.xlabel("rel. Train ID")
plt.ylabel("Cell ID")
plt.show()

In [ ]:
plt.figure(figsize=(10,5), dpi=100)
plt.title("All hits (blue) vs. %d ring-like hits (red)" %rings.sum())
plt.plot(train-train[0], score, marker='.', color='lightblue', lw=0, ms=2, alpha=0.5)
plt.plot(trainid_rings-train[0], scores_rings, marker='.', color='r', lw=0, ms=1, alpha=1)
plt.semilogy()
plt.xlabel("rel. Train ID")
plt.ylabel("Nr. of lit pixel")
plt.show()

In [ ]:
first  = np.argsort(nrphot_rings)[::-1][:50]
second = np.argsort(nrphot_rings)[::-1][50:100]
third  = np.argsort(nrphot_rings)[::-1][100:150]

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=10, figsize=(10,5), dpi=300)
fig.suptitle("Ring-like features - strongest (0-49)", y=0.95)
cmap = cm.get_cmap('inferno')
cmap.set_bad('0.75')
cmap.set_under('1.0')
for j in range(50):
    c,r = j//10, j%10
    axes[c,r].set_xticks([])
    axes[c,r].set_yticks([])
    axes[c,r].set_title("nphot=%d" %(nrphot_rings[first[j]]), color='0.5', fontsize=5, pad=2)
    mimg = np.ma.masked_array(np.float64(images_rings[first[j]]), mask=1-det.mask)
    mimg.data[mimg.data==0] = 1e-10
    axes[c,r].imshow(mimg, norm=colors.LogNorm(vmin=1, vmax=100), cmap=cmap)
#plt.savefig("../../plots/r%04d_best_shots.png" %run, bbox_inches='tight')
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=10, figsize=(10,5), dpi=300)
fig.suptitle("Ring-like features - strongest (50-99)", y=0.95)
cmap = cm.get_cmap('inferno')
cmap.set_bad('0.75')
cmap.set_under('1.0')
for j in range(50):
    c,r = j//10, j%10
    axes[c,r].set_xticks([])
    axes[c,r].set_yticks([])
    axes[c,r].set_title("nphot=%d" %(nrphot_rings[second[j]]), color='0.5', fontsize=5, pad=2)
    mimg = np.ma.masked_array(np.float64(images_rings[second[j]]), mask=1-det.mask)
    mimg.data[mimg.data==0] = 1e-10
    axes[c,r].imshow(mimg, norm=colors.LogNorm(vmin=1, vmax=100), cmap=cmap)
#plt.savefig("../../plots/r%04d_best_shots.png" %run, bbox_inches='tight')
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=10, figsize=(10,5), dpi=300)
fig.suptitle("Ring-like features - strongest (100-149)", y=0.95)
cmap = cm.get_cmap('inferno')
cmap.set_bad('0.75')
cmap.set_under('1.0')
for j in range(50):
    c,r = j//10, j%10
    axes[c,r].set_xticks([])
    axes[c,r].set_yticks([])
    axes[c,r].set_title("nphot=%d" %(nrphot_rings[third[j]]), color='0.5', fontsize=5, pad=2)
    mimg = np.ma.masked_array(np.float64(images_rings[third[j]]), mask=1-det.mask)
    mimg.data[mimg.data==0] = 1e-10
    axes[c,r].imshow(mimg, norm=colors.LogNorm(vmin=1, vmax=100), cmap=cmap)
#plt.savefig("../../plots/r%04d_best_shots.png" %run, bbox_inches='tight')
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=10, figsize=(10,5), dpi=300)
fig.suptitle("Ring-like features - weakest (last 50)", y=0.95)
cmap = cm.get_cmap('inferno')
cmap.set_bad('0.75')
cmap.set_under('1.0')
for j in range(50):
    c,r = j//10, j%10
    axes[c,r].set_xticks([])
    axes[c,r].set_yticks([])
    axes[c,r].set_title("nphot=%d" %(nrphot_rings[weak[j]]), color='0.5', fontsize=5, pad=2)
    mimg = np.ma.masked_array(np.float64(images_rings[weak[j]]), mask=1-det.mask)
    mimg.data[mimg.data==0] = 1e-10
    axes[c,r].imshow(mimg, norm=colors.LogNorm(vmin=1, vmax=100), cmap=cmap)
#plt.savefig("../../plots/r%04d_best_shots.png" %run, bbox_inches='tight')
plt.show()

In [ ]:
rmax = 150
radialavg = np.zeros((len(nrphot_rings),rmax))
index = np.argsort(nrphot_rings)[::-1]
qr = np.round(np.sqrt(det.qx**2 + det.qy**2))
for j in range(len(nrphot_rings)):
    for r in range(rmax):
        radialavg[j,r] = modules_rings[index[j]][(qr == r) & (det.raw_mask == 0)].mean()

In [ ]:
radialavg[radialavg==0] = 1e-5
radialavg[np.isnan(radialavg)] = 1e-5

In [ ]:
distance = 0.705 #[m]
wavelength = 1240. / 6010. #[nm]
pixelsize = 200e-6 #[m]
q_to_x = lambda q: np.tan(2.*np.arcsin(wavelength * q / 2.)) * distance / pixelsize
x_to_q = lambda x: (2*np.sin(np.arctan2(x*pixelsize, distance)/2.)) / wavelength 

In [ ]:
plt.figure(figsize=(15,5), dpi=100)
plt.imshow(radialavg[:], aspect='auto', norm=colors.LogNorm(vmin=0.01, vmax=5), cmap='cividis', interpolation='nearest')
plt.xticks(range(120)[::20], labels=["%.2f" %(x_to_q(i)/2.) for i in range(120)[::20]])
plt.axvline(46, color='r', ls=':')
plt.text(46,-10,"%.2f nm" %(1./x_to_q(46)/2.), ha='center', color='r')
plt.xlabel(r'Radial spatial frequency [nm$^{-1}$]')
plt.ylabel("ring-like events sorted by total nr. photons")
plt.colorbar(label='Nr. of photons')
plt.show()

In [ ]:
plt.figure(figsize=(10,2))
plt.title("Average over first 500 events")
plt.plot(x_to_q(np.arange(150))/2., radialavg[:500].mean(axis=0))
plt.ylim(0,0.5)
plt.xlim(0,x_to_q(150)/2.)
plt.axvline(x_to_q(46)/2., color='r', ls=':')
plt.xlabel(r'Radial spatial frequency [nm$^{-1}$]')
plt.show()

In [ ]:
plt.figure(figsize=(10,2))
plt.title("Average over last 500 events")
plt.plot(x_to_q(np.arange(150))/2., radialavg[-500:].mean(axis=0))
plt.ylim(0,0.05)
plt.xlim(0,x_to_q(150)/2.)
plt.axvline(x_to_q(46)/2., color='r', ls=':')
plt.xlabel(r'Radial spatial frequency [nm$^{-1}$]')
plt.show()